## Introduction
In this tutorial, we are going to perform Part-of-Speech (POS) tagging using Hidden Markov Models (HMM) and the Viterbi algorithm. POS tagging is the task of labeling the words in a sentence with their appropriate Part of Speech.

**Hidden Markov Models (HMMs)** are statistical models where the system being modeled is assumed to be a Markov process with hidden states. An HMM can be visualized as a directed, weighted graph, where each edge's weight represents the probability of transitioning between two states. You can see a [visual representation of an HMM here](https://en.wikipedia.org/wiki/File:HMMGraph.svg).

The **Viterbi algorithm** is a dynamic programming algorithm for finding the most likely sequence of hidden states—called the Viterbi path—that results in a sequence of observed events. It uses the concept of path and state probabilities. You can see a [step-by-step example of the Viterbi algorithm here](https://www.cis.upenn.edu/~cis262/notes/Example-Viterbi-DNA.pdf).

In [6]:
[# Necessary imports for the notebook
import numpy as np
from collections import defaultdict

At the beginning, we import the necessary libraries for this tutorial. numpy is used for handling operations on large, multi-dimensional arrays and matrices, and defaultdict is a dictionary subclass that provides a default value for the key that does not exists.

### Loading and Preprocessing Data

In [2]:
# Function to load and preprocess the data
def load_data(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    data = []
    for line in lines:
        line = line.strip()
        if line:  # Skip empty lines
            word, tag = line.split('\t')
            data.append((word, tag))
    
    return data

train_data = load_data('pos_data/train.pos')
test_data = load_data('pos_data/test.pos')

In [10]:
train_data[0:5]

[('In', 'IN'), ('an', 'DT'), ('Oct.', 'NNP'), ('19', 'CD'), ('review', 'NN')]

In this block of code, we're defining a function **load_data()** that will be used to load and preprocess the dataset. The function reads the file line by line, and for each line, it extracts the word and its corresponding tag, which are appended as a tuple to the data list. The function then returns the data list. We use this function to load our training and test data.

### Calculating Emission and Transition Probabilities

In [18]:
# Calculate emission probabilities
def emission_probabilities(data):
    emission_counts = defaultdict(lambda: defaultdict(int))
    tag_counts = defaultdict(int)
    
    for word, tag in data:
        emission_counts[tag][word] += 1
        tag_counts[tag] += 1

    # Calculate probabilities
    emission_probs = defaultdict(dict)
    for tag in emission_counts.keys():
        for word in emission_counts[tag].keys():
            emission_probs[tag][word] = emission_counts[tag][word] / tag_counts[tag]
    
    return emission_probs


# Calculate transition probabilities
def transition_probabilities(data):
    transition_counts = defaultdict(lambda: defaultdict(int))
    tag_counts = defaultdict(int)
    
    for i in range(len(data) - 1):
        current_tag = data[i][1]
        next_tag = data[i + 1][1]
        transition_counts[current_tag][next_tag] += 1
        tag_counts[current_tag] += 1

    # Calculate probabilities
    transition_probs = defaultdict(dict)
    for tag in transition_counts.keys():
        for next_tag in transition_counts[tag].keys():
            transition_probs[tag][next_tag] = transition_counts[tag][next_tag] / tag_counts[tag]
    
    return transition_probs

Next, we define two functions **emission_probabilities()** and **transition_probabilities()** that calculate the emission and transition probabilities respectively.

**Emission probability** is the probability of an observation being generated from a state. In our case, it's the probability of a word given its POS tag. It's calculated by dividing the count of a particular word-tag pair by the count of the tag.

**Transition probability** is the probability of transitioning from one state to another. In our case, it's the probability of a particular POS tag given the previous POS tag. It's calculated by dividing the count of a tag following a particular tag by the count of the latter tag.


### Implementing the Viterbi Algorithm

In [12]:
def viterbi(words, tags, start_probs, transition_probs, emission_probs):
    V = [{}]
    path = {}

    # Initialize base cases (t == 0)
    for tag in tags:
        V[0][tag] = start_probs[tag] * emission_probs[tag].get(words[0], 0)
        path[tag] = [tag]

    # Run Viterbi when t > 0
    for t in range(1, len(words)):
        V.append({})
        new_path = {}

        for current_tag in tags:
            (prob, state) = max((V[t-1][previous_tag] * transition_probs[previous_tag].get(current_tag, 0) * emission_probs[current_tag].get(words[t], 0), previous_tag) for previous_tag in tags)
            V[t][current_tag] = prob
            new_path[current_tag] = path[state] + [current_tag] 

        path = new_path
    
    (prob, state) = max((V[-1][current_tag], current_tag) for current_tag in tags)
    return (prob, path[state])

Here we implement the Viterbi algorithm. It's a dynamic programming algorithm used to find the most likely sequence of hidden states, given a sequence of observations. It returns the most probable sequence of tags for a given sequence of words.

### Putting It All Together

In [19]:
# Get unique words and tags from the training data
words = list(set([word for word, tag in train_data]))
tags = list(set([tag for word, tag in train_data]))

# Compute start, transition, and emission probabilities
start_probs = {tag: 1/len(tags) for tag in tags}  # Assume uniform distribution for start probabilities
transition_probs = transition_probabilities(train_data)
emission_probs = emission_probabilities(train_data)

# Implement Viterbi on the test data
predicted_tags = []
for word, tag in test_data:
    prob, path = viterbi([word], tags, start_probs, transition_probs, emission_probs)
    predicted_tags.append(path[0])

# Calculate accuracy
correct_tags = [tag for word, tag in test_data]
accuracy = sum([correct == predicted for correct, predicted in zip(correct_tags, predicted_tags)]) / len(correct_tags)
print('Accuracy:', accuracy)

Accuracy: 0.891973335768423


In this last block of code, we bring all the pieces together.

1. Extract unique words and tags from the training data.
2. Compute the start, transition, and emission probabilities.
3. Implement the Viterbi algorithm on the test data to predict the POS tags.
4. Calculate the accuracy of our predictions by comparing with the actual tags.

The output of the cell shows the accuracy of our POS tagging, which is approximately 0.89, indicating that our HMM model with the Viterbi algorithm is performing quite well.